In [ ]:
# Data Source
# 1. CIFAR-10: https://www.cs.toronto.edu/~kriz/cifar.html Accessed on 2024 11 11
#   - Files: data_batch_1, data_batch_2, data_batch_3, data_batch_4, data_batch_5, test_batch
#   - Renamed to: cifar10-train-1, cifar10-train-2, cifar10-train-3, cifar10-train-4, cifar10-train-5, cifar10-test

# 2. Fashion-MNIST: https://github.com/zresearch/fashion-mnist Accessed on 2024 11 11
#   - Files: train-images-idx3-ubyte, train-labels-idx1-ubyte, t10k-images-idx3-ubyte, t10k-labels-idx1-ubyte
#   - Renamed to: fmnist-train-image, fmnist-train-label, fmnist-test-image, fmnist-test-label

In [ ]:
# Importing Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import setuptools.dist  # To avoid tensorflow import error for distutils, requires setuptools to be installed
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# ==================================================================================================== Step 1. Loading Data
# 1. CIFAR-10 Ref: https://www.cs.toronto.edu/~kriz/cifar.html
# CIFAR-10 Class Label
cifar10_classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]


# For loading pickle files. From https://www.cs.toronto.edu/~kriz/cifar.html
def unpickle(file):
    import pickle

    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


# Data Frame for CIFAR-10
cifar10_train_data = []
cifar10_train_label = []

cifar10_test_data = []
cifar10_test_label = []


# 1.1 Load Train Data
for i in range(1, 6):
    file = f"./data/cifar10-train-{i}"
    cifar10_test_file = unpickle(file)
    cifar10_train_data.append(cifar10_test_file[b"data"])
    cifar10_train_label.append(cifar10_test_file[b"labels"])

cifar10_train_data = np.concatenate(cifar10_train_data)
cifar10_train_label = np.concatenate(cifar10_train_label)


# 1.2 Load Test Data
cifar10_test_file = unpickle("./data/cifar10-test")
cifar10_test_data.append(cifar10_test_file[b"data"])
cifar10_test_label.append(cifar10_test_file[b"labels"])
cifar10_test_data = np.concatenate(cifar10_test_data)
cifar10_test_label = np.concatenate(cifar10_test_label)


# 2. Fashion-MNIST Ref: https://pjreddie.com/projects/mnist-in-csv/ , https://www.kaggle.com/code/pavansanagapati/a-simple-cnn-model-beginner-guide
# Fashion MNIST Class Label
fmnist_classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]


# For converting idx to csv
def idx2csv(image_file, label_file, output_file):
    with open(image_file, "rb") as f:
        image_data = f.read()

    with open(label_file, "rb") as f:
        label_data = f.read()

    # Skip header
    image_data = image_data[16:]
    label_data = label_data[8:]
    image_length = len(image_data) // 784  # 28x28 = 784
    label_length = len(label_data)  # 1 byte per label
    images = np.zeros((image_length, 784), dtype=np.uint8)  # 28x28 = 784
    labels = np.zeros((label_length), dtype=np.uint8)  # 1 byte per label

    # i = image index, j = pixel index
    for i in range(image_length):
        for j in range(784):
            images[i, j] = image_data[i * 784 + j]  # 28x28 = 784

    # i = label index
    for i in range(label_length):
        labels[i] = label_data[i]  # 1 byte per label

    # Save to csv
    np.savetxt(
        output_file,
        np.concatenate((labels.reshape(-1, 1), images), axis=1),
        delimiter=",",
        fmt="%d",
    )


# 2.1 Convert idx to csv
if not os.path.exists("./data/fmnist-train.csv"):
    idx2csv("./data/fmnist-train-image", "./data/fmnist-train-label", "./data/fmnist-train.csv")
if not os.path.exists("./data/fmnist-test.csv"):
    idx2csv("./data/fmnist-test-image", "./data/fmnist-test-label", "./data/fmnist-test.csv")

# 2.2 Load Data
fmnist_train_data = pd.read_csv("./data/fmnist-train.csv", header=None).loc[:, 1:].values
fmnist_train_label = pd.read_csv("./data/fmnist-train.csv", header=None).loc[:, 0].values
fmnist_test_data = pd.read_csv("./data/fmnist-test.csv", header=None).loc[:, 1:].values
fmnist_test_label = pd.read_csv("./data/fmnist-test.csv", header=None).loc[:, 0].values

# 3. Augment Data
cifar10_train_data_augmented = []
cifar10_train_label_augmented = []
fmnist_train_data_augmented = []
fmnist_train_label_augmented = []

augment_count = 3

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",
)

# CIFAR-10
index = 0
for img in cifar10_train_data:
    a = img.reshape(3, 32, 32).transpose(1, 2, 0)
    a = a.reshape(1, 32, 32, 3)
    generator = data_generator.flow(a, batch_size=1)
    for i in range(augment_count):  # Generate 10 augmented images
        augmented_img = next(generator)[0]
        # Transpose it back to 3x32x32 from 4D tensor
        augmented_img = augmented_img.transpose(2, 0, 1)
        cifar10_train_data_augmented.append(augmented_img.round().astype(int).reshape(3 * 32 * 32))
        cifar10_train_label_augmented.append(cifar10_train_label[index])

    cifar10_train_data_augmented.append(img)
    cifar10_train_label_augmented.append(cifar10_train_label[index])
    index += 1

# Fashion MNIST
index = 0
for img in fmnist_train_data:
    a = img.reshape(28, 28)
    a = a.reshape(1, 28, 28, 1)
    generator = data_generator.flow(a, batch_size=1)
    # Get current image label
    for i in range(augment_count):  # Generate 10 augmented images
        augmented_img = next(generator)[0]
        fmnist_train_data_augmented.append(augmented_img.round().astype(int).reshape(28 * 28))
        fmnist_train_label_augmented.append(fmnist_train_label[index])

    fmnist_train_data_augmented.append(img)
    fmnist_train_label_augmented.append(fmnist_train_label[index])
    index += 1

# Convert to numpy array
cifar10_train_data_augmented = np.array(cifar10_train_data_augmented)
cifar10_train_label_augmented = np.array(cifar10_train_label_augmented)
fmnist_train_data_augmented = np.array(fmnist_train_data_augmented)
fmnist_train_label_augmented = np.array(fmnist_train_label_augmented)

# 4. Display Data
# 4.1 Display Data Shape
print("++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10")
print("Train Data | Label :", cifar10_train_data.shape, " | ", cifar10_train_label.shape)
print("Train Data Augmented | Label :", cifar10_train_data_augmented.shape, " | ", cifar10_train_label_augmented.shape)
print("Test Data | Label :", cifar10_test_data.shape, " | ", cifar10_test_label.shape)
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST")
print("Train Data | Label :", fmnist_train_data.shape, " | ", fmnist_train_label.shape)
print("Train Data Augmented | Label :", fmnist_train_data_augmented.shape, " | ", fmnist_train_label_augmented.shape)
print("Test Data | Label :", fmnist_test_data.shape, " | ", fmnist_test_label.shape)

# 4.2 Display image with label and class

print("++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10")
plt.figure(figsize=(12, 7))
plt.title("CIFAR-10 Train Data")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(cifar10_train_data[i].reshape(3, 32, 32).transpose(1, 2, 0))
    plt.xlabel(cifar10_classes[cifar10_train_label[i]])
plt.savefig("./figures/1 cifar10_train_data.jpg", dpi=600)
plt.show()

plt.figure(figsize=(12, 7))
plt.title("CIFAR-10 Test Data")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(cifar10_test_data[i].reshape(3, 32, 32).transpose(1, 2, 0))
    plt.xlabel(cifar10_classes[cifar10_test_label[i]])
plt.savefig("./figures/1 cifar10_test_data.jpg", dpi=600)
plt.show()

plt.figure(figsize=(12, 7))
plt.title("CIFAR-10 Augmented Train Data")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(cifar10_train_data_augmented[i].reshape(3, 32, 32).transpose(1, 2, 0))
    plt.xlabel(cifar10_classes[cifar10_train_label_augmented[i]])
plt.savefig("./figures/1 cifar10_train_data_augmented.jpg", dpi=600)
plt.show()

# Fashion MNIST
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST")
plt.figure(figsize=(12, 7))
plt.title("Fashion MNIST Train Data")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(fmnist_train_data[i].reshape(28, 28), cmap="gray")
    plt.xlabel(fmnist_classes[fmnist_train_label[i]])
plt.savefig("./figures/1 fmnist_train_data.jpg", dpi=600)
plt.show()

plt.figure(figsize=(12, 7))
plt.title("Fashion MNIST Test Data")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(fmnist_test_data[i].reshape(28, 28), cmap="gray")
    plt.xlabel(fmnist_classes[fmnist_test_label[i]])
plt.savefig("./figures/1 fmnist_test_data.jpg", dpi=600)
plt.show()

plt.figure(figsize=(12, 7))
plt.title("Fashion MNIST Augmented Train Data")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(fmnist_train_data_augmented[i].reshape(28, 28), cmap="gray")
    plt.xlabel(fmnist_classes[fmnist_train_label_augmented[i]])
plt.savefig("./figures/1 fmnist_train_data_augmented.jpg", dpi=600)
plt.show()

In [ ]:
# ==================================================================================================== Step 2. Data Preprocessing
# 1. CIFAR-10
# 1.1 Normalization
cifar10_train_data_normed = cifar10_train_data / 255.0
cifar10_test_data_normed = cifar10_test_data / 255.0
cifar10_train_data_augmented_normed = cifar10_train_data_augmented / 255.0
# 1.2 Reshape
cifar10_train_data_normed = cifar10_train_data_normed.reshape(-1, 32, 32, 3)  # 3 for RGB
cifar10_test_data_normed = cifar10_test_data_normed.reshape(-1, 32, 32, 3)  # 3 for RGB
cifar10_train_data_augmented_normed = cifar10_train_data_augmented_normed.reshape(-1, 32, 32, 3)  # 3 for RGB

# 2. Fashion-MNIST
# 2.1 Normalization
fmnist_train_data_normed = fmnist_train_data / 255.0
fmnist_test_data_normed = fmnist_test_data / 255.0
fmnist_train_data_augmented_normed = fmnist_train_data_augmented / 255.0

# 2.2 Reshape
fmnist_train_data_normed = fmnist_train_data_normed.reshape(-1, 28, 28, 1)  # 1 for grayscale
fmnist_test_data_normed = fmnist_test_data_normed.reshape(-1, 28, 28, 1)  # 1 for grayscale
fmnist_train_data_augmented_normed = fmnist_train_data_augmented_normed.reshape(-1, 28, 28, 1)  # 1 for grayscale


# 3. Check data shape before and after normalization
print("++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10")
print("Train Original Shape:", cifar10_train_data.shape, " | Normalized Shape:", cifar10_train_data_normed.shape)
print("Test Original Shape:", cifar10_test_data.shape, " | Normalized Shape:", cifar10_test_data_normed.shape)
print("Train Augmented Original Shape:", cifar10_train_data_augmented.shape, " | Normalized Shape:", cifar10_train_data_augmented_normed.shape)
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST")
print("Train Original Shape:", fmnist_train_data.shape, " | Normalized Shape:", fmnist_train_data_normed.shape)
print("Test Original Shape:", fmnist_test_data.shape, " | Normalized Shape:", fmnist_test_data_normed.shape)
print("Train Augmented Original Shape:", fmnist_train_data_augmented.shape, " | Normalized Shape:", fmnist_train_data_augmented_normed.shape)

# 4. Verify content of data
print("++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10 - Train Data")
print(cifar10_train_data[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10 - Test Data")
print(cifar10_test_data[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10 - Augmented Train Data")
print(cifar10_train_data_augmented[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10 - Train Data Normalized")
print(cifar10_train_data_normed[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10 - Test Data Normalized")
print(cifar10_test_data_normed[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10 - Augmented Train Data Normalized")
print(cifar10_train_data_augmented_normed[0])

print("++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST - Train Data")
print(fmnist_train_data[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST - Test Data")
print(fmnist_test_data[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST - Augmented Train Data")
print(fmnist_train_data_augmented[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST - Train Data Normalized")
print(fmnist_train_data_normed[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST - Test Data Normalized")
print(fmnist_test_data_normed[0])
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST - Augmented Train Data Normalized")
print(fmnist_train_data_augmented_normed[0])

In [ ]:
# Model architecture
# 1. Basic architecture
# Input Layer > Conv2D > MaxPooling2D > Conv2D > MaxPooling2D > Flatten > Dense > Output Layer

# 2. Model architecture from Kaggle
# CIFAR-10: https://www.kaggle.com/code/roblexnana/cifar10-with-cnn-for-beginer
# Model: Conv2D > Conv2D > MaxPooling2D > Dropout > Conv2D > Conv2D > MaxPooling2D > Dropout > Flatten > Dense > Dropout > Dense

# CIFAR-10: https://www.kaggle.com/code/ektasharma/simple-cifar10-cnn-keras-code-with-88-accuracy/notebook
# Model: Conv2D > BatchNorm > Conv2D > BatchNorm > MaxPooling2D > Dropout > Conv2D > BatchNorm > Conv2D > BatchNorm > MaxPooling2D > Dropout > Conv2D > BatchNorm > Conv2D > BatchNorm > MaxPooling2D > Dropout > Flatten > Dense > BatchNorm > Dropout > Dense

# Fashion MNIST: https://www.kaggle.com/code/pavansanagapati/a-simple-cnn-model-beginner-guide
# Model: Conv2D > MaxPooling2D > Dropout > Flatten > Dense > Dense

# Fashion MNIST: https://www.kaggle.com/code/franklemuchahary/basic-cnn-keras-with-cross-validation
# Model: Conv2D > conv2D > MaxPooling2D > Flatten > Dense > Dropout > Dense > Dense

# 3. Proposed Model - basic
# Model: Conv2D > MaxPooling2D > Conv2D > MaxPooling2D > Flatten > Dense > Dense

# 4. Proposed Model - advanced (with BatchNormalization and Dropout)
# Model: Conv2D > BatchNorm > Conv2D > BatchNorm > MaxPooling2D > Dropout > Conv2D > BatchNorm > Conv2D > BatchNorm > MaxPooling2D > Dropout > Conv2D > BatchNorm > Conv2D > BatchNorm > MaxPooling2D > Dropout > Flatten > Dense > BatchNorm > Dropout > Dense

# 5. Proposed Model - ultimate (advanced model with k fold cross validation and data augmentation)
# Model: Conv2D > BatchNorm > Conv2D > BatchNorm > MaxPooling2D > Dropout > Conv2D > BatchNorm > Conv2D > BatchNorm > MaxPooling2D > Dropout > Conv2D > BatchNorm > Conv2D > BatchNorm > MaxPooling2D > Dropout > Flatten > Dense > BatchNorm > Dropout > Dense

# Method for addressing Underfitting and Overfitting
# 1. General: Learning rate, epochs
# 2. Underfitting: Data augmentation (rotation, flip, zoom, shift)
# 3. Overfitting handling: Dropout, l2 regularization, BatchNormalization

In [ ]:
# ==================================================================================================== Step 3. Model Building
# 1. Model Building
# 1.1 Basic Model
def build_basic_model(input_shape, num_classes):
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation="relu", input_shape=input_shape),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return model


# 1.2 Advanced Model
def build_advanced_model(input_shape, num_classes):
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation="relu", input_shape=input_shape),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return model


# 1.3 Ultimate Model
def build_ultimate_model(input_shape, num_classes):
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation="relu", input_shape=input_shape),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return model


# 2. Model Compilation
# 2.1 Basic Model
basic_model_cifar = build_basic_model((32, 32, 3), 10)
basic_model_cifar.compile(optimizer="adam", loss="SparseCategoricalCrossentropy", metrics=["accuracy"])
basic_model_fmnist = build_basic_model((28, 28, 1), 10)
basic_model_fmnist.compile(optimizer="adam", loss="SparseCategoricalCrossentropy", metrics=["accuracy"])

# 2.2 Advanced Model
advanced_model_cifar = build_advanced_model((32, 32, 3), 10)
advanced_model_cifar.compile(optimizer="adam", loss="SparseCategoricalCrossentropy", metrics=["accuracy"])
advanced_model_fmnist = build_advanced_model((28, 28, 1), 10)
advanced_model_fmnist.compile(optimizer="adam", loss="SparseCategoricalCrossentropy", metrics=["accuracy"])

# 2.3 Ultimate Model
ultimate_model_cifar = build_ultimate_model((32, 32, 3), 10)
ultimate_model_cifar.compile(optimizer="adam", loss="SparseCategoricalCrossentropy", metrics=["accuracy"])
ultimate_model_fmnist = build_ultimate_model((28, 28, 1), 10)
ultimate_model_fmnist.compile(optimizer="adam", loss="SparseCategoricalCrossentropy", metrics=["accuracy"])

# 2.3 Model Summary
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Basic Model - CIFAR")
basic_model_cifar.summary()
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Basic Model - Fashion MNIST")
basic_model_fmnist.summary()
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Advanced Model - CIFAR")
advanced_model_cifar.summary()
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Advanced Model - Fashion MNIST")
advanced_model_fmnist.summary()
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Ultimate Model - CIFAR")
ultimate_model_cifar.summary()
print("++++++++++++++++++++++++++++++++++++++++++++++++++ Ultimate Model - Fashion MNIST")
ultimate_model_fmnist.summary()

In [ ]:
# ==================================================================================================== Step 4. Model Training
# 1. Set early stopping and checkpoint
n_folds = 5
epochs = 100
batch_size = 64
model_hist_cifar10_basic = []
model_hist_cifar10_advance = []
model_hist_cifar10_ultimate = []
model_hist_fmnist_basic = []
model_hist_fmnist_advance = []
model_hist_fmnist_ultimate = []

pat = 90  # Number of epochs with no improvement after which training will be stopped
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

# Set checkpoint
cifar10_basic_chkpt_path = "./model/cifar10_basic_model.keras"
cifar10_advance_chkpt_path = "./model/cifar10_advance_model.keras"
cifar10_ultimate_chkpt_path = "./model/cifar10_ultimate_model.keras"
cifar10_basic_cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cifar10_basic_chkpt_path, save_best_only=True, verbose=1)
cifar10_advance_cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cifar10_advance_chkpt_path, save_best_only=True, verbose=1)
cifar10_ultimage_cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cifar10_ultimate_chkpt_path, save_best_only=True, verbose=1)
fmnist_basic_chkpt_path = "./model/fmnist_basic_model.keras"
fmnist_advance_chkpt_path = "./model/fmnist_advance_model.keras"
fminst_ultimate_chkpt_path = "./model/fmnist_ultimate_model.keras"
fmnist_basic_cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=fmnist_basic_chkpt_path, save_best_only=True, verbose=1)
fmnist_advance_cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=fmnist_advance_chkpt_path, save_best_only=True, verbose=1)
fmnist_ultimate_cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=fminst_ultimate_chkpt_path, save_best_only=True, verbose=1)


# Define fit function
def fit_and_evaluate(model, model_checkpoint, train_data, train_label, test_data, test_label, epochs, batch_size):
    history = model.fit(train_data, train_label, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, model_checkpoint], verbose=1, validation_split=0.1)
    print("Validation Accuracy: ", model.evaluate(test_data, test_label))
    return history


# 2. Model Training
# 1.1 Basic Model
print("Training Basic Model ++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10")
# model_hist_cifar10_basic.append(fit_and_evaluate(basic_model_cifar, cifar10_basic_cp_callback, cifar10_train_data_normed, cifar10_train_label, cifar10_test_data_normed, cifar10_test_label, epochs, batch_size))
print("Training Basic Model ++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST")
# model_hist_fmnist_basic.append(fit_and_evaluate(basic_model_fmnist, fmnist_basic_cp_callback, fmnist_train_data_normed, fmnist_train_label, fmnist_test_data_normed, fmnist_test_label, epochs, batch_size))

# 1.2 Advanced Model
print("Training Advanced Model ++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10")
# model_hist_cifar10_advance.append(fit_and_evaluate(advanced_model_cifar, cifar10_advance_cp_callback, cifar10_train_data_normed, cifar10_train_label, cifar10_test_data_normed, cifar10_test_label, epochs, batch_size))
print("Training Advanced Model ++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST")
# model_hist_fmnist_advance.append(fit_and_evaluate(advanced_model_fmnist, fmnist_advance_cp_callback, fmnist_train_data_normed, fmnist_train_label, fmnist_test_data_normed, fmnist_test_label, epochs, batch_size))


# 1.3 Ultimate Model
print("Training Ultimate Model ++++++++++++++++++++++++++++++++++++++++++++++++++ CIFAR-10")
for i in range(n_folds):
    print("++++++++++++++++++++++++++++++++++++++++++++++++++ Training CIFAR-10 on Fold: ", i + 1)
    t_x, val_x, t_y, val_y = train_test_split(cifar10_train_data_augmented_normed, cifar10_train_label_augmented, test_size=0.1, random_state=np.random.randint(1, 1000, 1)[0])
    model_hist_cifar10_ultimate.append(fit_and_evaluate(ultimate_model_cifar, cifar10_ultimage_cp_callback, t_x, t_y, val_x, val_y, epochs, batch_size))
    print("END ==========================================================================")

# Fashion MNIST
print("Training Ultimate Model ++++++++++++++++++++++++++++++++++++++++++++++++++ Fashion MNIST")
for i in range(n_folds):
    print("Training Fashion MNIST on Fold: ", i + 1)
    t_x, val_x, t_y, val_y = train_test_split(fmnist_train_data_augmented_normed, fmnist_train_label_augmented, test_size=0.1, random_state=np.random.randint(1, 1000, 1)[0])
    model_hist_fmnist_ultimate.append(fit_and_evaluate(ultimate_model_fmnist, fmnist_ultimate_cp_callback, t_x, t_y, val_x, val_y, epochs, batch_size))
    print("END ==========================================================================")

# 3. Save Model
basic_model_cifar.save("./model/cifar10_basic_model.h5")
basic_model_fmnist.save("./model/fmnist_basic_model.h5")
advanced_model_cifar.save("./model/cifar10_advance_model.h5")
advanced_model_fmnist.save("./model/fmnist_advance_model.h5")
ultimate_model_cifar.save("./model/cifar10_ultimate_model.h5")
ultimate_model_fmnist.save("./model/fmnist_ultimate_model.h5")

In [ ]:
# ==================================================================================================== Step 5. Model Evaluation
# 1. Model Evaluation
# 1.1 Basic Model
color = ["black", "red", "green", "blue", "orange"]
# CIFAR-10
plt.title("Basic Model - CIFAR-10 - Train Accuracy vs Val Accuracy")
plt.plot(model_hist_cifar10_basic[0].history['accuracy'], label='train', color=color[0])
plt.plot(model_hist_cifar10_basic[0].history['val_accuracy'], label='val', color=color[1])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig("./figures/2 basic_model_cifar10_accuracy.jpg", dpi=600)
plt.show()
# Fashion MNIST
plt.title("Basic Model - Fashion MNIST - Train Accuracy vs Val Accuracy")
plt.plot(model_hist_fmnist_basic[0].history['accuracy'], label='train', color=color[0])
plt.plot(model_hist_fmnist_basic[0].history['val_accuracy'], label='val', color=color[1])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig("./figures/2 basic_model_fmnist_accuracy.jpg", dpi=600)
plt.show()

# 1.2 Advanced Model
# CIFAR-10
plt.title("Advanced Model - CIFAR-10 - Train Accuracy vs Val Accuracy")
plt.plot(model_hist_cifar10_advance[0].history['accuracy'], label='train', color=color[0])
plt.plot(model_hist_cifar10_advance[0].history['val_accuracy'], label='val', color=color[1])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig("./figures/2 advanced_model_cifar10_accuracy.jpg", dpi=600)
plt.show()
# Fashion MNIST
plt.title("Advanced Model - Fashion MNIST - Train Accuracy vs Val Accuracy")
plt.plot(model_hist_fmnist_advance[0].history['accuracy'], label='train', color=color[0])
plt.plot(model_hist_fmnist_advance[0].history['val_accuracy'], label='val', color=color[1])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig("./figures/2 advanced_model_fmnist_accuracy.jpg", dpi=600)
plt.show()

# 1.3 Ultimate Model
# CIFAR-10
plt.title("Ultimate Model - CIFAR-10 - Train Accuracy vs Val Accuracy")
for i in range(n_folds):
    plt.plot(model_hist_cifar10_ultimate[i].history['accuracy'], label='train', color=color[i])
    plt.plot(model_hist_cifar10_ultimate[i].history['val_accuracy'], label='val', color=color[i + 1])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig("./figures/2 ultimate_model_cifar10_accuracy.jpg", dpi=600)
plt.show()
# Fashion MNIST
plt.title("Ultimate Model - Fashion MNIST - Train Accuracy vs Val Accuracy")
for i in range(n_folds):
    plt.plot(model_hist_fmnist_ultimate[i].history['accuracy'], label='train', color=color[i])
    plt.plot(model_hist_fmnist_ultimate[i].history['val_accuracy'], label='val', color=color[i + 1])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig("./figures/2 ultimate_model_fmnist_accuracy.jpg", dpi=600)
plt.show()

In [ ]:
# ==================================================================================================== Step 6. Model Prediction
# 1. Model Prediction

# Uncomment to load model for prediction if not trained
# basic_model_cifar = tf.keras.models.load_model("./model/cifar10_basic_model.h5")
# basic_model_fmnist = tf.keras.models.load_model("./model/fmnist_basic_model.h5")
# advanced_model_cifar = tf.keras.models.load_model("./model/cifar10_advance_model.h5")
# advanced_model_fmnist = tf.keras.models.load_model("./model/fmnist_advance_model.h5")
# ultimate_model_cifar = tf.keras.models.load_model("./model/cifar10_ultimate_model.h5")
# ultimate_model_fmnist = tf.keras.models.load_model("./model/fmnist_ultimate_model.h5")

# 1.1 Basic Model
cifar10_test_prediction_basic = basic_model_cifar.predict(cifar10_test_data_normed)
fmnist_test_prediction_basic = basic_model_fmnist.predict(fmnist_test_data_normed)

# 1.2 Advanced Model
cifar10_test_prediction_advanced = advanced_model_cifar.predict(cifar10_test_data_normed)
fmnist_test_prediction_advanced = advanced_model_fmnist.predict(fmnist_test_data_normed)

# 1.3 Ultimate Model
cifar10_test_prediction_ultimate = ultimate_model_cifar.predict(cifar10_test_data_normed)
fmnist_test_prediction_ultimate = ultimate_model_fmnist.predict(fmnist_test_data_normed)

In [ ]:
# ==================================================================================================== Step 7. Prediction Evaluation
# 1. Error Analysis
# for heatmap and confusion matrix
def heatmap(data, row_lbl, col_lbl, ax=None, cbar_kw={}, cbarlabel="", **kwargs):
    if not ax:
        ax = plt.gca()
    # Plot the heatmap
    im = ax.imshow(data, **kwargs)
    # Create color bar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries
    ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))

    ax.set_xticklabels(col_lbl)
    ax.set_yticklabels(row_lbl)

    # set x and y labels
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')

    return im, cbar


# to annotate the heatmap
def annotate_heatmap(im, data=None, fmt="d", threshold=None):
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            text = im.axes.text(j, i, format(data[i, j], fmt), horizontalalignment="center", color="white" if data[i, j] > threshold else "black")
            texts.append(text)

    return texts


# 1.1 Basic Model
# CIFAR-10
cifar10_test_prediction_basic_label = np.argmax(cifar10_test_prediction_basic, axis=1)
cifar10_test_confusion_matrix_basic = confusion_matrix(cifar10_test_label, cifar10_test_prediction_basic_label)
plt.figure(figsize=(10, 7))
sns.heatmap(cifar10_test_confusion_matrix_basic, annot=True, fmt="d", cmap="Reds", xticklabels=cifar10_classes, yticklabels=cifar10_classes)
plt.title("Basic Model - CIFAR-10 - Prediction Confusion Matrix")
plt.savefig("./figures/3 cifar10_basic_prediction_confusion_matrix.jpg", dpi=600)
plt.show()

# Fashion MNIST
fmnist_test_prediction_basic_label = np.argmax(fmnist_test_prediction_basic, axis=1)
fmnist_test_confusion_matrix_basic = confusion_matrix(fmnist_test_label, fmnist_test_prediction_basic_label)
plt.figure(figsize=(10, 7))
sns.heatmap(fmnist_test_confusion_matrix_basic, annot=True, fmt="d", cmap="Reds", xticklabels=fmnist_classes, yticklabels=fmnist_classes)
plt.title("Basic Model - Fashion MNIST - Prediction Confusion Matrix")
plt.savefig("./figures/3 fmnist_basic_prediction_confusion_matrix.jpg", dpi=600)
plt.show()

# 1.2 Advanced Model
# CIFAR-10
cifar10_test_prediction_advanced_label = np.argmax(cifar10_test_prediction_advanced, axis=1)
cifar10_test_confusion_matrix_advanced = confusion_matrix(cifar10_test_label, cifar10_test_prediction_advanced_label)
plt.figure(figsize=(10, 7))
sns.heatmap(cifar10_test_confusion_matrix_advanced, annot=True, fmt="d", cmap="Reds", xticklabels=cifar10_classes, yticklabels=cifar10_classes)
plt.title("Advanced Model - CIFAR-10 - Prediction Confusion Matrix")
plt.savefig("./figures/3 cifar10_advanced_confusion_matrix.jpg", dpi=600)
plt.show()

# Fashion MNIST
fmnist_test_prediction_advanced_label = np.argmax(fmnist_test_prediction_advanced, axis=1)
fmnist_test_confusion_matrix_advanced = confusion_matrix(fmnist_test_label, fmnist_test_prediction_advanced_label)
plt.figure(figsize=(10, 7))
sns.heatmap(fmnist_test_confusion_matrix_advanced, annot=True, fmt="d", cmap="Reds", xticklabels=fmnist_classes, yticklabels=fmnist_classes)
plt.title("Advanced Model - Fashion MNIST - Prediction Confusion Matrix")
plt.savefig("./figures/3 fmnist_advanced_confusion_matrix.jpg", dpi=600)
plt.show()

# 1.3 Ultimate Model
# CIFAR-10
cifar10_test_prediction_ultimate_label = np.argmax(cifar10_test_prediction_ultimate, axis=1)
cifar10_test_confusion_matrix_ultimate = confusion_matrix(cifar10_test_label, cifar10_test_prediction_ultimate_label)
plt.figure(figsize=(10, 7))
sns.heatmap(cifar10_test_confusion_matrix_ultimate, annot=True, fmt="d", cmap="Reds", xticklabels=cifar10_classes, yticklabels=cifar10_classes)
plt.title("Ultimate Model - CIFAR-10 - Prediction Confusion Matrix")
plt.savefig("./figures/3 cifar10_ultimate_confusion_matrix.jpg", dpi=600)
plt.show()

# Fashion MNIST
fmnist_test_prediction_ultimate_label = np.argmax(fmnist_test_prediction_ultimate, axis=1)
fmnist_test_confusion_matrix_ultimate = confusion_matrix(fmnist_test_label, fmnist_test_prediction_ultimate_label)
plt.figure(figsize=(10, 7))
sns.heatmap(fmnist_test_confusion_matrix_ultimate, annot=True, fmt="d", cmap="Reds", xticklabels=fmnist_classes, yticklabels=fmnist_classes)
plt.title("Ultimate Model - Fashion MNIST - Prediction Confusion Matrix")
plt.savefig("./figures/3 fmnist_ultimate_confusion_matrix.jpg", dpi=600)
plt.show()

# 2. Display Prediction
# 2.1 Basic Model
# CIFAR-10
plt.figure(figsize=(30, 15))
plt.title("Basic Model - CIFAR-10")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(cifar10_test_data[i].reshape(3, 32, 32).transpose(1, 2, 0))
    plt.xlabel(f"Ori: {cifar10_classes[cifar10_test_label[i]]} - Pre: {cifar10_classes[np.argmax(cifar10_test_prediction_basic[i])]}")
plt.savefig("./figures/4 cifar10_basic_prediction.jpg", dpi=600)
plt.show()

# Fashion MNIST
plt.figure(figsize=(30, 15))
plt.title("Basic Model - Fashion MNIST")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(fmnist_test_data[i].reshape(28, 28), cmap="gray")
    plt.xlabel(f"Ori: {fmnist_classes[fmnist_test_label[i]]} - Pre: {fmnist_classes[np.argmax(fmnist_test_prediction_basic[i])]}")
plt.savefig("./figures/4 fmnist_basic_prediction.jpg", dpi=600)
plt.show()

# 2.2 Advanced Model
# CIFAR-10
plt.figure(figsize=(30, 15))
plt.title("Advanced Model - CIFAR-10")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(cifar10_test_data[i].reshape(3, 32, 32).transpose(1, 2, 0))
    plt.xlabel(f"Ori: {cifar10_classes[cifar10_test_label[i]]} - Pre: {cifar10_classes[np.argmax(cifar10_test_prediction_advanced[i])]}")
plt.savefig("./figures/4 cifar10_advanced_prediction.jpg", dpi=600)
plt.show()

# Fashion MNIST
plt.figure(figsize=(30, 15))
plt.title("Advanced Model - Fashion MNIST")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(fmnist_test_data[i].reshape(28, 28), cmap="gray")
    plt.xlabel(f"Ori: {fmnist_classes[fmnist_test_label[i]]} - Pre: {fmnist_classes[np.argmax(fmnist_test_prediction_advanced[i])]}")
plt.savefig("./figures/4 fmnist_advanced_prediction.jpg", dpi=600)
plt.show()

# 2.3 Ultimate Model
# CIFAR-10
plt.figure(figsize=(30, 15))
plt.title("Ultimate Model - CIFAR-10")
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(cifar10_test_data[i].reshape(3, 32, 32).transpose(1, 2, 0))
    plt.xlabel(f"Ori: {cifar10_classes[cifar10_test_label[i]]} - Pre: {cifar10_classes[np.argmax(cifar10_test_prediction_ultimate[i])]}")
plt.savefig("./figures/4 cifar10_ultimate_prediction.jpg", dpi=600)
plt.show()